# Structure
1. classic pipeline, params hardcoded everywhere
2. parametrizable function, introduce DictConfig
3. hydra decorates, introduce configuration file
4. introduce override syntax commandline
5. introduce sweeps (e.g. multiple seeds), sequential
6. introduce launchers (local: joblib, but also slurm/submitit)
7. introduce composition
8. introduce instantiation

# AutoML Fall School 2023 Hydra Hands-On

Welcome to our tutorial session on [hydra](hydra.cc)! 🐍
Hydra is a tool for configuring and running your experiments and optimization is seemlessly integrated.

Hydra can:

* Hierarchical configuration composable from multiple sources
* Configuration can be specified or overridden from the command line
* Dynamic command line tab completion
* Run your application locally or launch it to run remotely
* Run multiple jobs with different arguments with a single command



## A Classical Training Pipeline

(This part of the tutorial is the same as in the SMAC tutorial).

We'll start with your classical optimization task.
The task is to optimize the hyperparameters of a [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) on the [digits](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) dataset. Usually we have some training pipeline with a dataset, a configured model[^1] and some validation procedure to check for generalization performance like this:


[^1]: If we check out the documentation, we will see that loads of design decisions (hyperparameters) are already set to a default value for us.

In [1]:
from IPython.display import Code
Code(filename="hydra_tutorial/classic_pipeline_hardcoded.py")

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# We load the digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

def train_mlp() -> float:
    # we want to have reproducible training
    np.random.seed(seed=1234)

    # for illustrative purposes, you can reduce max_iter drastically here
    classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, activation='relu', solver='adam')
    scores = cross_val_score(classifier, X_train, y_train, cv=5)

    return np.mean(scores) # mean accuracy over folds

if __name__ == "__main__":
    # Ignore the warnings for now:)
    cv_loss = train_mlp()
    print(f'Cross_validation accuaracy on digits {cv_loss}')

In [2]:
import subprocess
subprocess.run("python hydra_tutorial/classic_pipeline_hardcoded.py".split(" "))

Cross_validation accuaracy on digits 0.9625795297372062


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline_hardcoded.py'], returncode=0)


You can ignore the errors above regarding not converging for now.

What we can see in this example is that we hardcoded many (hyper-)parameters. But maybe we would like to vary them? So let's adapt our `train_mlp` function!
We will use a dict-like object to hold all our parameters.

In [3]:
Code(filename="hydra_tutorial/classic_pipeline_stillhardcoded.py")

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from omegaconf import DictConfig
from rich import print as printr
from rich import inspect
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# We load the digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


def train_mlp(cfg: DictConfig) -> float:
    # we want to have reproducible training
    np.random.seed(seed=cfg.seed)

    # for illustrative purposes, you can reduce max_iter drastically here
    classifier = MLPClassifier(hidden_layer_sizes=cfg.hidden_layer_sizes, max_iter=cfg.max_iter, activation=cfg.activation, solver=cfg.solver)
    scores = cross_val_score(classifier, X_train, y_train, cv=5)

    return np.mean(scores) # mean accuracy over folds

if __name__ == "__main__":
    # Ignore the warnings for now:)
    # We can easily create a DictConfig object with dict-like syntax
    # We can have almost any type in here
    cfg = DictConfig({
        "seed": 1234,
        "hidden_layer_sizes": (100,),
        "max_iter": 100,
        "activation": 'relu',
        "solver": 'adam',
    })
    inspect(cfg)
    
    cv_loss = train_mlp(cfg=cfg)
    print(f'Cross_validation accuaracy on digits {cv_loss}')

In [4]:
subprocess.run("python hydra_tutorial/classic_pipeline_stillhardcoded.py".split(" "))

╭───────────────── <class 'omegaconf.dictconfig.DictConfig'> ──────────────────╮
│ ╭──────────────────────────────────────────────────────────────────────────╮ │
│ │ {'seed': 1234, 'hidden_layer_sizes': [100], 'max_iter': 100,             │ │
│ │ 'activation': 'relu', 'solver': 'adam'}                                  │ │
│ ╰─────────────────────────��────────────────────────────────────────────────╯ │
│                                                                              │
│         activation = 'relu'                                                  │
│ hidden_layer_sizes = [100]                                                   │
│           max_iter = 100                                                     │
│               seed = 1234                                                    │
│             solver = 'adam'                                                  │
╰────────────────────────────��─────────────────────────────────────────────────╯
Cross_validation accuaracy

CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline_stillhardcoded.py'], returncode=0)

That's nice but let's vary the parameters with hydra!
Hydra can wrap your main function and pass parameters from the command line or configuration files for you -- without the hassle of writing an argument parser.

In [5]:
Code(filename="hydra_tutorial/classic_pipeline.py")

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from omegaconf import DictConfig
import hydra
from sklearn.exceptions import ConvergenceWarning
import warnings
from hydra_tutorial.utils import dump_logs

warnings.filterwarnings("ignore", category=ConvergenceWarning)

# We load the digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


@hydra.main(version_base=None, config_path="configs", config_name="base")
def train_mlp(cfg: DictConfig) -> float:
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    # we want to have reproducible training
    np.random.seed(seed=cfg.seed)

    # for illustrative purposes, you can reduce max_iter drastically here
    classifier = MLPClassifier(
        hidden_layer_sizes=cfg.hidden_layer_sizes, max_iter=cfg.max_iter, activation=cfg.activation, solver=cfg.solver,
        alpha=cfg.alpha)
    scores = cross_val_score(classifier, X_train, y_train, cv=5)

    mean_score = np.mean(scores)

    print(f"Mean accuracy: {mean_score:.4f}")

    # Let's produce a log file
    dump_logs(log_data={"mean_acc": mean_score}, filename="performance.jsonl")

    return mean_score # mean accuracy over folds

if __name__ == "__main__":
    train_mlp()

Before we let it run, let's have a look at the configuration file.
This will be our default as decorated by hydra.

In [6]:
Code(filename="hydra_tutorial/configs/base.yaml")

# @package _global_
defaults:
  - _self_
  - override hydra/launcher: joblib
  - override hydra/job_logging: colorlog
  - override hydra/hydra_logging: colorlog

seed: 1234
hidden_layer_sizes: [100,]
max_iter: 100
activation: 'relu'
solver: 'adam'
alpha: 0.0001

In [7]:
subprocess.run("python hydra_tutorial/classic_pipeline.py".split(" "))

Mean accuracy: 0.9626


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py'], returncode=0)

Now we want to pass arguments via the commandline.
This is easily possible with the [override syntax](https://hydra.cc/docs/advanced/override_grammar/basic/).
Let's vary the hidden layer sizes.

In [8]:
subprocess.run("python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[10,10,10]".split(" "))

Mean accuracy: 0.8945


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', 'hidden_layer_sizes=[10,10,10]'], returncode=0)

But what really comes in handy is the ability to grid of parameter settings.
First, to factor out randomness we want to run different seeds.
Then, we would like to check different settings, e.g. different hidden layer sizes and different activation functions.

For this we will add a list of parameter values and the flag `-m` or `--multirun` to indicate that this is a grid.

In [9]:
subprocess.run("python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[100],[10,10,10] seed=range(1,6) -m".split(" "))

[2023-11-21 18:16:26,808][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 10 jobs
[2023-11-21 18:16:26,809][HYDRA] Launching jobs, sweep output dir : multirun/2023-11-21/18-16-26
[2023-11-21 18:16:26,809][HYDRA] 	#0 : hidden_layer_sizes=[100] seed=1
[2023-11-21 18:16:26,809][HYDRA] 	#1 : hidden_layer_sizes=[100] seed=2
[2023-11-21 18:16:26,809][HYDRA] 	#2 : hidden_layer_sizes=[100] seed=3
[2023-11-21 18:16:26,809][HYDRA] 	#3 : hidden_layer_sizes=[100] seed=4
[2023-11-21 18:16:26,809][HYDRA] 	#4 : hidden_layer_sizes=[100] seed=5
[2023-11-21 18:16:26,809][HYDRA] 	#5 : hidden_layer_sizes=[10,10,10] seed=1
[2023-11-21 18:16:26,809][HYDRA] 	#6 : hidden_layer_sizes=[10,10,10] seed=2
[2023-11-21 18:16:26,809][HYDRA] 	#7 : hidden_layer_sizes=[10,10,10] seed=3
[2023-11-21 18:16:26,809][HYDRA] 	#8 : hidden_layer_sizes=[10,10,10] seed=4
[2023-11-21 18

Error executing job with overrides: ['hidden_layer_sizes=[100]', 'seed=1']
Traceback (most recent call last):
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/classic_pipeline.py", line 39, in train_mlp
    dump_logs(log_data={"mean_acc": mean_score}, filename="performance.jsonl")
      ^^^^^^^^^^^^^^^^^
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/utils.py", line 33, in dump_logs
    with open(filename, mode="a") as file:
      ^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2023-11-21/18-16-26/performance.jsonl'

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', 'hidden_layer_sizes=[100],[10,10,10]', 'seed=range(1,6)', '-m'], returncode=1)

You can specifiy the number of workers on your local cluster via the config files.
This config runs your parallel jobs with joblib.
The config might look like this:

In [10]:
Code(filename="hydra_tutorial/configs/cluster/local.yaml")

# @package _global_
defaults:
  - override /hydra/launcher: joblib

hydra:
  launcher:
    n_jobs: 4

In [11]:
subprocess.run("python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[100],[10,10,10] seed=range(1,6) +cluster=local -m".split(" "))

[2023-11-21 18:16:35,981][HYDRA] Joblib.Parallel(n_jobs=4,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 10 jobs
[2023-11-21 18:16:35,981][HYDRA] Launching jobs, sweep output dir : multirun/2023-11-21/18-16-35
[2023-11-21 18:16:35,981][HYDRA] 	#0 : hidden_layer_sizes=[100] seed=1 +cluster=local
[2023-11-21 18:16:35,981][HYDRA] 	#1 : hidden_layer_sizes=[100] seed=2 +cluster=local
[2023-11-21 18:16:35,981][HYDRA] 	#2 : hidden_layer_sizes=[100] seed=3 +cluster=local
[2023-11-21 18:16:35,982][HYDRA] 	#3 : hidden_layer_sizes=[100] seed=4 +cluster=local
[2023-11-21 18:16:35,982][HYDRA] 	#4 : hidden_layer_sizes=[100] seed=5 +cluster=local
[2023-11-21 18:16:35,982][HYDRA] 	#5 : hidden_layer_sizes=[10,10,10] seed=1 +cluster=local
[2023-11-21 18:16:35,982][HYDRA] 	#6 : hidden_layer_sizes=[10,10,10] seed=2 +cluster=local
[2023-11-21 18:16:35,982][HYDRA] 	#7 : hidden_layer_sizes=[10,

Error executing job with overrides: ['hidden_layer_sizes=[100]', 'seed=1', '+cluster=local']
Traceback (most recent call last):
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/classic_pipeline.py", line 39, in train_mlp
    dump_logs(log_data={"mean_acc": mean_score}, filename="performance.jsonl")
      ^^^^^^^^^^^^^^^^^
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/utils.py", line 33, in dump_logs
    with open(filename, mode="a") as file:
      ^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2023-11-21/18-16-35/performance.jsonl'

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', 'hidden_layer_sizes=[100],[10,10,10]', 'seed=range(1,6)', '+cluster=local', '-m'], returncode=1)

You can also dispatch parallel jobs on your slurm cluster with the following command:

`python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[100],[10,10,10] seed=range(1,6) +cluster=slurm -m`

and the corresponding config file looks like this:

In [12]:
Code(filename="hydra_tutorial/configs/cluster/slurm.yaml")

# @package _global_
defaults:
  - override /hydra/launcher: submitit_slurm

# Check more possible options here:
# https://hydra.cc/docs/plugins/submitit_launcher/
hydra:
  launcher:
    # partition: normal  # Set your partition here
    timeout_min: 10
    cpus_per_task: 1

Let's run! Here, we overwrite the launcher with the local version to test it, because we don't have a cluster at hand right now.
For this we append `hydra/launcher=submitit_local`.

In [13]:
subprocess.run("python hydra_tutorial/classic_pipeline.py hidden_layer_sizes=[100],[10,10,10] seed=range(1,6) +cluster=slurm hydra/launcher=submitit_local -m".split(" "))

[2023-11-21 18:16:49,420][HYDRA] Submitit 'local' sweep output dir : multirun/2023-11-21/18-16-48
[2023-11-21 18:16:49,421][HYDRA] 	#0 : hidden_layer_sizes=[100] seed=1 +cluster=slurm
[2023-11-21 18:16:49,426][HYDRA] 	#1 : hidden_layer_sizes=[100] seed=2 +cluster=slurm
[2023-11-21 18:16:49,430][HYDRA] 	#2 : hidden_layer_sizes=[100] seed=3 +cluster=slurm
[2023-11-21 18:16:49,435][HYDRA] 	#3 : hidden_layer_sizes=[100] seed=4 +cluster=slurm
[2023-11-21 18:16:49,441][HYDRA] 	#4 : hidden_layer_sizes=[100] seed=5 +cluster=slurm
[2023-11-21 18:16:49,445][HYDRA] 	#5 : hidden_layer_sizes=[10,10,10] seed=1 +cluster=slurm
[2023-11-21 18:16:49,450][HYDRA] 	#6 : hidden_layer_sizes=[10,10,10] seed=2 +cluster=slurm
[2023-11-21 18:16:49,455][HYDRA] 	#7 : hidden_layer_sizes=[10,10,10] seed=3 +cluster=slurm
[2023-11-21 18:16:49,460][HYDRA] 	#8 : hidden_layer_sizes=[10,10,10] seed=4 +cluster=slurm
[2023-11-21 18:16:49,465][HYDRA] 	#9 : hidden_layer_sizes=[10,10,10] seed=5 +cluster=slurm


Error executing job with overrides: ['hidden_layer_sizes=[100]', 'seed=1', '+cluster=slurm']
Traceback (most recent call last):
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/classic_pipeline.py", line 39, in train_mlp
    dump_logs(log_data={"mean_acc": mean_score}, filename="performance.jsonl")
      ^^^^^^^^^^^^^^^^^
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/utils.py", line 33, in dump_logs
    with open(filename, mode="a") as file:
      ^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2023-11-21/18-16-48/performance.jsonl'

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', 'hidden_layer_sizes=[100],[10,10,10]', 'seed=range(1,6)', '+cluster=slurm', 'hydra/launcher=submitit_local', '-m'], returncode=1)

## Composition
A great feature of hydra is that you can compose your configuration with different config files. We already did this with the cluster config, but here we do it more explicitly.
This comes especially in handy if we want to configure different modules and experiments.

In [14]:
# One composition
subprocess.run("python hydra_tutorial/classic_pipeline.py +architecture=tiny_mlp +solver=adam".split(" "))

# Sweep over all in one folder
subprocess.run("python hydra_tutorial/classic_pipeline.py +architecture=glob('*') +solver=adam -m".split(" "))

Mean accuracy: 0.9343
[2023-11-21 18:17:28,819][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 3 jobs
[2023-11-21 18:17:28,819][HYDRA] Launching jobs, sweep output dir : multirun/2023-11-21/18-17-28
[2023-11-21 18:17:28,819][HYDRA] 	#0 : +architecture=big_mlp +solver=adam
[2023-11-21 18:17:28,819][HYDRA] 	#1 : +architecture=medium_mlp +solver=adam
[2023-11-21 18:17:28,819][HYDRA] 	#2 : +architecture=tiny_mlp +solver=adam
Mean accuracy: 0.9343
Mean accuracy: 0.9626
Mean accuracy: 0.9717


Error executing job with overrides: ['+architecture=big_mlp', '+solver=adam']
Traceback (most recent call last):
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/classic_pipeline.py", line 39, in train_mlp
    dump_logs(log_data={"mean_acc": mean_score}, filename="performance.jsonl")
      ^^^^^^^^^^^^^^^^^
  File "/home/numina/Documents/repos/hydra_tutorial/hydra_tutorial/utils.py", line 33, in dump_logs
    with open(filename, mode="a") as file:
      ^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2023-11-21/18-17-28/performance.jsonl'

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', "+architecture=glob('*')", '+solver=adam', '-m'], returncode=1)

## Instantiation
Hydra can also instantiate classes from config files, either fully or partially. 
This means you have full flexibility of configuring your project!

Check the docs here: https://hydra.cc/docs/advanced/instantiate_objects/overview/

## Hyperparameter Optimization
We can not only run configurations in parallel, we can also do proper HPO with hydra with suitable plugins.
Available sweepers are:
- [Ax](https://hydra.cc/docs/plugins/ax_sweeper/)
- [Optuna](https://hydra.cc/docs/plugins/nevergrad_sweeper/)
- [Nevergrad](https://hydra.cc/docs/plugins/optuna_sweeper/)

In addition, we are happy to announce the first alpha version of the [Hydra-SMAC-Sweeper](https://github.com/automl/hydra-smac-sweeper)! 🥳
We have created an interface between hydra and [SMAC3](https://github.com/automl/SMAC3).
Make sure you installed everything via `bash install.sh`. 🙂

In [16]:
subprocess.run("python hydra_tutorial/classic_pipeline.py +hpo=smac -m".split(" "))

Config {'hydra': {'run': {'dir': 'outputs/${now:%Y-%m-%d}/${now:%H-%M-%S}'}, 
'sweep': {'dir': 'multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}', 'subdir': 
'${hydra.job.num}'}, 'launcher': {'_target_': 
'hydra_plugins.hydra_joblib_launcher.joblib_launcher.JoblibLauncher', 'n_jobs': 
-1, 'backend': None, 'prefer': 'processes', 'require': None, 'verbose': 0, 
'timeout': None, 'pre_dispatch': '2*n_jobs', 'batch_size': 'auto', 
'temp_folder': None, 'max_nbytes': None, 'mmap_mode': 'r'}, 'sweeper': 
{'_target_': 'hydra_plugins.hydra_smac_sweeper.smac_sweeper.SMACSweeper', 
'search_space': {'hyperparameters': {'alpha': {'type': 'uniform_float', 'lower':
1e-06, 'upper': 0.01, 'log': True, 'default_value': '${alpha}'}, 'activation': 
{'type': 'categorical', 'choices': ['logistic', 'tanh', 'relu'], 
'default_value': '${activation}'}}}, 'scenario': {'seed': '${seed}', 'n_trials':
100, 'deterministic': True, 'n_workers': 4}, 'smac_class': 
'smac.facade.blackbox_facade.BlackBoxFacade', 'smac_kwargs': {'

/home/numina/Documents/repos/hydra_tutorial/hydra-smac-sweeper/hydra_plugins/hydra_smac_sweeper/smac_sweeper_backend.py:291: UserWarning: Override arguments might not have an effect if they are a sweep. ['+hpo=smac']
  warnings.warn(f"Override arguments might not have an effect if they are a sweep. {arguments}")


[INFO][abstract_initial_design.py:147] Using 16 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
[INFO][smbo.py:319] Finished 0 trials.
Mean accuracy: 0.9584
Mean accuracy: 0.9609
[INFO][abstract_intensifier.py:515] Added config ec7abb as new incumbent because there are no incumbents yet.
[INFO][abstract_intensifier.py:590] Added config 311943 and rejected config ec7abb as incumbent because it is not better than the incumbents on 1 instances:
Mean accuracy: 0.9634
[INFO][abstract_intensifier.py:590] Added config 4486e7 and rejected config 311943 as incumbent because it is not better than the incumbents on 1 instances:
Mean accuracy: 0.9634
Mean accuracy: 0.9592
Mean accuracy: 0.9634
Mean accuracy: 0.9634
Mean accuracy: 0.9617
Mean accuracy: 0.9626
Mean accuracy: 0.9609
Mean 

CompletedProcess(args=['python', 'hydra_tutorial/classic_pipeline.py', '+hpo=smac', '-m'], returncode=0)